In [1]:
import os
import cv2
import torch
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
import albumentations as A
import torch.optim as optim
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm
from termcolor import colored
from matplotlib import patches
from collections import Counter
from torchvision import transforms
from torch.utils.data import DataLoader
from albumentations.pytorch import ToTensorV2

In [ ]:
def convert_to_yolo_format(target, img_width, img_height, class_mapping):
    annotations = target["annotation"]["object"]
    
    real_width = int(target["annotation"]["size"]["width"])
    real_height = int(target["annotation"]["size"]["height"])
    
    if not isinstance(annotations, list):
        annotations = [annotations]
        
    boxes = []
    
    for anno in annotations:
        xmin = int(anno["bndbox"]["xmin"]) / real_width
        xmax = int(anno["bndbox"]["xmax"]) / real_height
        ymin = int(anno["bndbox"]["ymin"]) / real_width
        ymax = int(anno["bndbox"]["ymax"]) / real_height
        
        x_center = (xmin + xmax) / 2
        y_canter = (ymin + ymax) / 2

In [ ]:
class CustomVOCDataset(torchvision.datasets.VOCDetection):
    def init_config_yolo(self, class_mapping, S=7, B=2, C=20, custom_transforms=None):
        self.S = S
        self.B = B
        self.C = C
        self.class_mapping = class_mapping
        self.custom_transforms = custom_transforms
        
    def __getitem__(self, index):
        image, target = super(CustomVOCDataset, self).__getitem__(index)
        img_width, img_height = image.size
        
        